# Playground

My playground, for learning / debugging / visualizing purposes

## Look up dataset images: in a simple way

In [ ]:
import os
from os import path as osp
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import ipywidgets as widgets

In [2]:
data_path = "./data/"
dataset_path = osp.join(data_path, 'CVACT_full', 'streetview')
dataset_filenames = sorted(os.listdir(dataset_path))

In [3]:
def show_img(dataset_path, filenames, idx):
    img_path = osp.join(dataset_path, filenames[idx])
    img = np.asarray(Image.open(img_path))
    print(f'\tFile path: {img_path}')
    print(f'\tImage shape: {img.shape}')
    plt.imshow(img)
    plt.show()

def look_dataset(dataset_path, filenames, s=0, l=100):
    print(f'Dataset path: {dataset_path}')
    print(f'Number of images in {dataset_path}: {len(dataset_filenames)}')
    print(f'Current visualizing range: {s} - {s + l}')
    widgets.interact(lambda idx: show_img(dataset_path, filenames, idx),
                     idx=widgets.IntSlider(min=s, max=min(len(dataset_filenames) - 1, s + l), step=1))

In [4]:
look_dataset(dataset_path, dataset_filenames, 300, 100)

Dataset path: ./data/CVACT_full/streetview
Number of images in ./data/CVACT_full/streetview: 128331
Current visualizing range: 300 - 400


interactive(children=(IntSlider(value=300, description='idx', max=400, min=300), Output()), _dom_classes=('wid…

## Look up CVACT dataset: interactively

In [1]:
from pathlib import Path
from orissl_cvm.datasets.cvact_dataset import CVACTDataset
from orissl_cvm.utils.tools import input_transform
from torch.utils.data import DataLoader

In [76]:
root_dir = Path('./data/CVACT_full/').absolute()

train_dataset = CVACTDataset(root_dir, 
                             mode='train', 
                             nNeg=3, 
                             transform=input_transform(),
                             bs=32, 
                             threads=6, 
                             margin=0.1)

Construct neighbor searches: kd_tree
Finding positive neighbors for 8646 queries
Finding non-negative neighbors for 8646 queries


In [77]:
# print(train_dataset.qIdx)
# print(train_dataset.pIdx)
# print(train_dataset.nonNegIdx)
# print(train_dataset.posDistThr)
# print(train_dataset.negDistThr)


In [78]:
from orissl_cvm.models.siamese import GoodNet
import torch

config = {
    'global_params': {'pooling': 'max'}
}
checkpoint = torch.load("work_dirs/Apr05_12-36-10_cvact_resnet50/checkpoints/checkpoint.pth.tar", map_location=lambda storage, loc: storage)
model = GoodNet(config['global_params'])
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [ ]:
# divides dataset into smaller cache sets
train_dataset.new_epoch()

# creates triplets on the smaller cache set
# train_dataset.update_subcache()
train_dataset.update_subcache(model, 512)

print(len(train_dataset.triplets))
print(train_dataset.triplets)

In [80]:
# create data loader
opt = {'batch_size': 16, 'shuffle': True, 'collate_fn': CVACTDataset.collate_fn}
training_loader = DataLoader(train_dataset, **opt)

In [81]:
import matplotlib.pyplot as plt
import numpy as np

def denormalize(im):
	image = im.numpy()
	im = (image - np.min(image)) / (np.max(image) - np.min(image))
	im = np.ascontiguousarray(im * 255, dtype=np.uint8)
	return im

def visualize_triplet(batch, sample_idx):

	query, negatives, meta = batch
	negCounts, indices, keys = meta['negCounts'], meta['indices'], meta['keys']

	B = query[0].shape[0]

	num_ns = negCounts[sample_idx].item()
	num_qns = num_ns + 1

	neg_start = 0
	start = 0
	if sample_idx > 0: 
		neg_start = negCounts[:sample_idx].sum().item()
		start = neg_start + sample_idx * 1
	# print(sample_idx, start)

	fig, axes = plt.subplots(nrows=num_qns, ncols=2, figsize=(15,9))
	fig.suptitle(
		f'Navigate dataloader of CVACT: current batch, sample {sample_idx} (1 query and {num_ns} negatives)',
		fontsize=15)
	fig.tight_layout()
	fig.subplots_adjust(top=0.9)
	
	axes[0,0].imshow(np.transpose(denormalize(query[0][sample_idx]),(1,2,0)))
	axes[0,0].set_title(
		f"Query ==> ground image\nidx: {indices[start]}, file name: {keys[sample_idx]['query']['gr_img']}")

	axes[0,1].imshow(np.transpose(denormalize(query[1][sample_idx]),(1,2,0)))
	axes[0,1].set_title(
		f"Query ==> satellite image\nidx: {indices[start]}, file name: {keys[sample_idx]['query']['sa_img']}")

	# axes[1,0].imshow(np.transpose(denormalize(positive[0][sample_idx]),(1,2,0)))
	# axes[1,0].set_title(
	# 	f"Positive ==> ground image\n{keys[sample_idx]['positive']['gr_img']}")
	
	# axes[1,1].imshow(np.transpose(denormalize(positive[1][sample_idx]),(1,2,0)))
	# axes[1,1].set_title(
	# 	f"Positive ==> satellite image\n{keys[sample_idx]['positive']['sa_img']}")

	for i in range(num_ns):
		axes[1+i,0].imshow(np.transpose(denormalize(negatives[0][neg_start+i]),(1,2,0)))
		axes[1+i,0].set_title(
			f"Negative {i} ==> ground image\nidx: {indices[start+i+1]}, file name: {keys[sample_idx]['negatives'][i]['gr_img']}")

		axes[1+i,1].imshow(np.transpose(denormalize(negatives[1][neg_start+i]),(1,2,0)))
		axes[1+i,1].set_title(
			f"Negative {i} ==> satellite image\nidx: {indices[start+i+1]}, file name: {keys[sample_idx]['negatives'][i]['sa_img']}")

	plt.show()

In [82]:
# visualize triplets

import ipywidgets as widgets
from IPython.display import display, clear_output

button = widgets.Button(
    description='Next Batch',
    layout=widgets.Layout(width='10%')
)
out = widgets.Output()
bs = training_loader.batch_size
it = iter(training_loader)

def on_button_clicked(_):
    # "linking function with output"
    with out:
        try:
            batch = next(it)
        except StopIteration:
            print("Data loader ran out.")
        clear_output()
        # display(f'')
        sample_slider = widgets.IntSlider(
            value=0, min=0, max=bs-1, step=1, 
            description='Sample:',
            layout=widgets.Layout(width='25%')
        )
        widgets.interact(lambda sample_idx: visualize_triplet(batch, sample_idx),
                         sample_idx=sample_slider)
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([button,out])
    

## Look up VGG16 architecture

In [65]:
import torchvision.models as models
import torch.nn as nn

In [2]:
enc = models.vgg16(pretrained=True)
print(enc)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
layers = list(enc.features.children())

In [21]:
print(len(layers))
for layer in layers[:-5]:
    print(layer)
    for p in layer.parameters():
        pass

In [77]:
enc = models.resnet50(pretrained=True)
layers = list(enc.children())
print(len(layers))
for layer in layers:
    print(layer)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/jianfengcui/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

10
Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU(inplace=True)
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True

In [83]:
layers = list(enc.children())[:-2]
backbone = nn.Sequential(*layers)
im = torch.rand(1, 3, 448, 448)
fmp = backbone(im)
print(fmp.shape)

torch.Size([1, 2048, 14, 14])


## Load the model and summarywriter

In [98]:
import torch
import os

from glob import glob
for path in glob("work_dirs/*"):
    checkpoint = torch.load(path + '/checkpoints/checkpoint.pth.tar', map_location=lambda storage, loc: storage)
    print(path.split('/')[1], '\t\t', checkpoint['best_score'])

May03-1816_cvm_vgg16_max_shared_cvact_5k 		 0.6815
May01-2242_cvm_vgg16_safa_shared_cvact_5k 		 0.847
May03-2019_cvm_vgg16_max_noshared_cvact_1k 		 0.274
May03-0045_simsiam_vgg16_max_shared_cvact_mini 		 -1.7663664247492235
May01-2233_simsiam_vgg16_max_satellite_cvact_mini 		 -0.9577470079814787
May03-2108_oripred_identity_identity_shared_cvact_1k 		 0
May03-2126_oripred_vgg16_max_shared_cvact_1k 		 0
May01-2242_cvm_vgg16_safa_noshared_cvact_5k 		 0.758
May02-0042_cvm_vit_identity_noshared_cvact_5k 		 0.0035
May03-1943_cvm_vgg16_max_shared_cvact_1k 		 0.56
May03-2053_oripred_resnet18_identity_shared_cvact_1k 		 0
May01-2206_cvm_vgg16_max_shared_cvact_5k 		 0.718
May01-2219_cvm_vgg16_max_noshared_cvact_5k 		 0.7065
May01-2233_simsiam_vgg16_max_ground_cvact_mini 		 -0.9095116543898479
May02-2156_cvm_vgg16_max_noshared_cvact_5k 		 0.6375
May03-1745_cvm_resnet18_max_shared_cvact_5k 		 0.1165
May03-0159_simsiam_vgg16_safa_shared_cvact_mini 		 -1.8826354594110586
May03-1944_cvm_vgg16_max_sha

In [12]:
checkpoint = torch.load('work_dirs/Apr28_21-47-32_simsiam_vgg16_max_cvact_ground/checkpoints/checkpoint.pth.tar', map_location=lambda storage, loc: storage)
print(checkpoint.keys())
print(checkpoint['state_dict'].keys())

dict_keys(['epoch', 'state_dict', 'best_score', 'not_improved', 'optimizer', 'parallel'])
odict_keys(['backbone.0.weight', 'backbone.0.bias', 'backbone.2.weight', 'backbone.2.bias', 'backbone.5.weight', 'backbone.5.bias', 'backbone.7.weight', 'backbone.7.bias', 'backbone.10.weight', 'backbone.10.bias', 'backbone.12.weight', 'backbone.12.bias', 'backbone.14.weight', 'backbone.14.bias', 'backbone.17.weight', 'backbone.17.bias', 'backbone.19.weight', 'backbone.19.bias', 'backbone.21.weight', 'backbone.21.bias', 'backbone.24.weight', 'backbone.24.bias', 'backbone.26.weight', 'backbone.26.bias', 'backbone.28.weight', 'backbone.28.bias', 'projector.layer1.0.weight', 'projector.layer1.0.bias', 'projector.layer1.1.weight', 'projector.layer1.1.bias', 'projector.layer1.1.running_mean', 'projector.layer1.1.running_var', 'projector.layer1.1.num_batches_tracked', 'projector.layer2.0.weight', 'projector.layer2.0.bias', 'projector.layer2.1.weight', 'projector.layer2.1.bias', 'projector.layer2.1.runni

In [1]:
from torch.utils.tensorboard import SummaryWriter

## Pretrain

In [1]:
from __future__ import print_function

import argparse
import yaml
from easydict import EasyDict as edict  
import os
import random
from os.path import join, isfile
from os import makedirs
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim

from tensorboardX import SummaryWriter
import numpy as np
from torch.utils.data import DataLoader

from orissl_cvm import PACKAGE_ROOT_DIR
from orissl_cvm.datasets.cvact_dataset_pre import CVACTDatasetPretrain
from orissl_cvm.tools.pretrain_epoch import pretrain_epoch
from orissl_cvm.tools.val import val
from orissl_cvm.tools import save_checkpoint
from orissl_cvm.utils import input_transform
from orissl_cvm.datasets.cvact_dataset import CVACTDataset
from orissl_cvm.models.safa import SAFAvgg16Cls
from orissl_cvm.tools.visualize import visualize_dataloader

from tqdm.auto import trange


# Load config file
cfg_file = "configs/train.yaml"
assert os.path.isfile(cfg_file)
with open(cfg_file, 'r') as f:
    config = edict(yaml.load(f, Loader=yaml.Loader))

print('===> Config file')

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"] = config.train.gpu_ids
os.environ["MKL_NUM_THREADS"] = config.train.threads
os.environ["NUMEXPR_NUM_THREADS"] = config.train.threads
os.environ["OMP_NUM_THREADS"] = config.train.threads

# CUDA setting
cuda = not config.train.no_cuda
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run with --nocuda")

device = torch.device("cuda" if cuda else "cpu")
print(f'num of cuda device: {torch.cuda.device_count()}')
print(f'current cuda device: {torch.cuda.current_device()}')

# Random seeds
random.seed(config.train.seed)
np.random.seed(config.train.seed)
torch.manual_seed(config.train.seed)
if cuda:
    # noinspection PyUnresolvedReferences
    torch.cuda.manual_seed(config.train.seed)

# Model: with resuming or not
print('===> Building model')

if config.train.resume_path: # if already started training earlier and continuing
    if isfile(config.train.resume_path):
        print("=> loading checkpoint '{}'".format(config.train.resume_path))
        checkpoint = torch.load(config.train.resume_path, map_location=lambda storage, loc: storage)

        model = SAFAvgg16Cls()

        model.load_state_dict(checkpoint['state_dict'])
        config.train.start_epoch = checkpoint['epoch']

        print("=> loaded checkpoint '{}'".format(config.train.resume_path))
    else:
        raise FileNotFoundError("=> no checkpoint found at '{}'".format(config.train.resume_path))
else: # if not, assume fresh training instance and will initially generate cluster centroids
    print('===> Loading model')
    model = SAFAvgg16Cls()

desc_dim = model.SAFAvgg16.desc_dim
print("===> Model")
print(model)

# If DataParallel
# TODO learn more about multi-gpu training. Actually more stuff needs
# to be considered, e.g., only log info of local rank 0
isParallel = False
if config.train.n_gpu > 1 and torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
    isParallel = True

# Optimizer and scheduler
optimizer = None
scheduler = None

if config.params.optim == 'ADAM':
    optimizer = optim.Adam(filter(lambda par: par.requires_grad,
                                    model.parameters()), lr=config.params.lr)  # , betas=(0,0.9))
elif config.params.optim == 'SGD':
    optimizer = optim.SGD(filter(lambda par: par.requires_grad,
                                    model.parameters()), lr=config.params.lr,
                            momentum=config.params.momentum,
                            weight_decay=config.params.weight_decay)

    # TODO include scheduler later
    # scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=config.train.lr_step,
    #                                       gamma=config.train.lr_gamma)
else:
    raise ValueError('Unknown optimizer: ' + config.params.optim)

model = model.to(device)

if config.train.resume_path:
    optimizer.load_state_dict(checkpoint['optimizer'])

===> Config file
num of cuda device: 1
current cuda device: 0
===> Building model
===> Loading model
===> Model
SAFAvgg16Cls(
  (SAFAvgg16): SAFAvgg16(
    (nn_model_gr): Module(
      (encoder): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU(inplace=True)
        (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (6): ReLU(inplace=True)
        (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): ReLU(inplace=True)
        (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): ReLU(inplace=True)
        (12): Conv2d(256, 256, kernel_size=(3, 3), stri

In [2]:
# Loss
# TODO delete it later, because we're actually using our own loss function
criterion = nn.CrossEntropyLoss().to(device)

# Dataset and dataloader
print('===> Loading dataset(s)')
train_dataset = CVACTDatasetPretrain(config.train.dataset_root_dir, 
                                    mode='train', 
                                    transform=input_transform())
print(f'Full num of images in training set: {train_dataset.qImages.shape[0]}')
print(f'Num of queries in training set: {len(train_dataset)}')

training_data_loader = DataLoader(dataset=train_dataset, 
    num_workers=int(config.train.threads),
    batch_size=config.train.batch_size, 
    shuffle=True,
    pin_memory=cuda
)

===> Loading dataset(s)
Construct neighbor searches for train set: kd_tree
Finding positive neighbors for 5404 queries
Finding non-negative neighbors for 5404 queries
Full num of images in training set: 5404
Num of queries in training set: 5404


In [4]:
# NOTE visualize batches for debug
visualize_dataloader(training_data_loader)

ValueError: too many values to unpack (expected 4)

## zip

In [31]:
import numpy as np

a1 = [np.zeros(1)]
a2 = a1 * 2
b1 = [np.ones(1)]
b2 = b1 * 2

print(a2)
print(b2)

[array([0.]), array([0.])]
[array([1.]), array([1.])]


In [35]:
c = 'token3'
d = 'token4'
for x, y in zip([a2, b2], [c, d]):
    print(x, y)
for x, y in zip([a1, a2, b1, b2], [c, d]):
    print(x, y)
# 直观来说，zip起来直接一起iterate更方便

[array([0.]), array([0.])] token3
[array([1.]), array([1.])] token4
[array([0.])] token3
[array([0.]), array([0.])] token4


In [36]:
uppercase = ['A', 'B', 'C']
lowercase = ['a', 'b', 'c']
numbers = [1, 2, 3]

for x, y, z in zip(uppercase, lowercase, numbers):
    print(x, y, z)

A a 1
B b 2
C c 3


## Others

In [3]:
import numpy as np
a = np.array([[1, 2, 3], [4, 5, 6]])
a[..., :0]

array([], shape=(2, 0), dtype=int64)

## Correlation

In [62]:
def horizontal_correlation(fmp1, fmp2):
    B, C, H = fmp1.shape[:3]
    print(B, C, H)
    W1, W2 = fmp1.shape[3], fmp2.shape[3]
    fmp2 = torch.cat([fmp2, fmp2[..., :(W1 - 1)]], dim=-1)
    resp = torch.empty((B, 1, 1, W2))
    # NOTE the batch dim of the kernel fmp will serve as the num of kernels.
    # so now do it separately
    for i in range(B):
        resp[i:i+1] = F.conv2d(fmp2[i:i+1], fmp1[i:i+1], bias=None, stride=1, padding=0)
    resp = resp.flatten(start_dim=-3, end_dim=-1)
    _, resp = torch.max(resp, dim=1, keepdim=True)
    return resp

In [63]:
import torch
import torch.nn.functional as F

a = torch.tensor([[1, 2, 3, 4], [5, 6, 7, 8]], dtype=float)
b = torch.tensor([[2, 3, 4, 1], [6, 7, 8, 5]], dtype=float)
c = torch.tensor([[3, 4, 1, 2], [7, 8, 5, 6]], dtype=float)
d = torch.tensor([[4, 1, 2, 3], [8, 5, 6, 7]], dtype=float)
a = a[None, None, ...]
b = b[None, None, ...]
c = c[None, None, ...]
d = d[None, None, ...]
print(horizontal_correlation(a, a))
print(horizontal_correlation(b, a))
print(horizontal_correlation(c, a))
print(horizontal_correlation(d, a))
print(horizontal_correlation(d, b))

1 1 2
tensor([[0]])
1 1 2
tensor([[1]])
1 1 2
tensor([[2]])
1 1 2
tensor([[3]])
1 1 2
tensor([[2]])
